Technex MLWare entry 

by Ayush Agarwal and Anshuman Asuliya 

team andrew ng sir team 

In [ ]:
# install kaggle
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anshumanasauliya","key":"49364d517a0a12c374d75c7e1c6f9350"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle competitions download -c ml-ware22

100% 2.48G/2.49G [00:24<00:00, 75.6MB/s]
100% 2.49G/2.49G [00:25<00:00, 107MB/s] 
100% 1.22G/1.23G [00:11<00:00, 167MB/s]
100% 1.23G/1.23G [00:11<00:00, 119MB/s]
  0% 0.00/44.0k [00:00<?, ?B/s]
100% 44.0k/44.0k [00:00<00:00, 28.1MB/s]


In [ ]:
!unzip test.h5.zip
!unzip train.h5.zip

Archive:  test.h5.zip
  inflating: test.h5                 
Archive:  train.h5.zip
  inflating: train.h5                


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np 
import pandas as pd
import h5py

ds = h5py.File('train.h5' , 'r')
x = (ds['x'])
y = ds['y']

# image after image processing
X_train = []
Y_train = []

In [ ]:
# this was also one of our codes which we used before , this uses image processing concepts and then uses rmsprop blob detection
# alogrithms , however we got a slightly higher accuracy with CNN so we used cnn in the end 
import random
for j in range(0,3000):

  # selecting random image
  i=random.randrange(0,x.shape[0])

  # taking the not of the image 
  img1 = cv2.bitwise_not(x[i,:,:,2])

  # applying the required threshold
  ret, thresh1 = cv2.threshold(img1, 140, 255, cv2.THRESH_TOZERO)

  # blackening the borders 
  shapea = thresh1.shape
  for a in range(0, shapea[0]):
    for b in range(0, 15):
            thresh1[a,b] = 0
  for a in range(0, shapea[0]):
    for b in range(shapea[1]-15,shapea[1]):
            thresh1[a,b] = 0
  for a in range(0, 15):
    for b in range(0, shapea[1]):
            thresh1[a,b] = 0
  for a in range(shapea[0]-15, shapea[0]):
    for b in range(0,shapea[1]):
            thresh1[a,b] = 0 
  # cv2_imshow(cv2.resize(thresh1,(0,0),fx=0.5,fy=0.5))

  ret, thresh2 = cv2.threshold(thresh1, 110, 255, cv2.THRESH_BINARY)
  # appending normalized image 
  X_train.append((thresh2/255).astype(np.float16))
  Y_train.append(y[i])

In [ ]:
# converting to numpy
X_train = np.array(X_train)
# reshaping test data
print(X_train.shape)

Y_train = np.array(Y_train)
print(Y_train.shape)

(3000, 299, 299)
(3000,)


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# dividing test data in batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(2000).batch(1000)

In [ ]:
# The CNN model which was used for the final predictions 
# model
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(299, 299, 1)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='relu')
])

In [ ]:
# compiling model
cnn.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [ ]:
# training model of all batched 
for images,label in train_ds:
  cnn.fit(images , label , epochs=10)

Epoch 1/10
32/32 [==============================] - 15s 156ms/step - loss: 30.1073 - accuracy: 0.2370
Epoch 2/10
32/32 [==============================] - 5s 145ms/step - loss: 30.5036 - accuracy: 0.2340
Epoch 3/10
32/32 [==============================] - 5s 148ms/step - loss: 30.4089 - accuracy: 0.2480
Epoch 4/10
32/32 [==============================] - 5s 146ms/step - loss: 17.6721 - accuracy: 0.2990
Epoch 5/10
32/32 [==============================] - 5s 144ms/step - loss: 8.8596 - accuracy: 0.2770
Epoch 6/10
32/32 [==============================] - 5s 145ms/step - loss: 2.9866 - accuracy: 0.3420
Epoch 7/10
32/32 [==============================] - 5s 145ms/step - loss: 2.0057 - accuracy: 0.3560
Epoch 8/10
32/32 [==============================] - 5s 145ms/step - loss: 0.9417 - accuracy: 0.3790
Epoch 9/10
32/32 [==============================] - 5s 145ms/step - loss: 0.6727 - accuracy: 0.4150
Epoch 10/10
32/32 [==============================] - 5s 145ms/step - loss: 0.5613 - accuracy: 0

In [ ]:
# test data
test = h5py.File('test.h5' , 'r')
x_test = test['x']

#test data after threshold
X_test = []

In [ ]:
for i in range(0,x_test.shape[0]):

  # #taking the not of the image 
  img1 = cv2.bitwise_not(x_test[i,:,:,2])

  # #applying the required threshold
  ret, thresh1 = cv2.threshold(img1, 140, 255, cv2.THRESH_TOZERO)

  # blackening the borders 
  shapea = thresh1.shape
  for a in range(0, shapea[0]):
    for b in range(0, 15):
            thresh1[a,b] = 0
  for a in range(0, shapea[0]):
    for b in range(shapea[1]-15,shapea[1]):
            thresh1[a,b] = 0
  for a in range(0, 15):
    for b in range(0, shapea[1]):
            thresh1[a,b] = 0
  for a in range(shapea[0]-15, shapea[0]):
    for b in range(0,shapea[1]):
            thresh1[a,b] = 0

  ret, thresh2 = cv2.threshold(thresh1, 110, 255, cv2.THRESH_BINARY)
  # appending normalized image in data list
  X_test.append((thresh2/255).astype(np.float16))

In [ ]:
# convert ti numpy array
X_test = np.array(X_test)
# reshaping test data
X_test = np.reshape(X_test , (X_test.shape[0] , 299,299,1))
print(X_test.shape)


(6600, 299, 299, 1)


In [ ]:
# prediction
y_pred = cnn.predict(X_test)

In [ ]:
# converting to numpy
y_pred = np.array(y_pred)
print(y_pred.shape)

(6600, 1)


In [ ]:
# round off predected values
y_pred = np.round(y_pred)
# reshaping predicted data
y_pred = y_pred.flatten()
print(y_pred)

[ 2.  2.  1. ...  1.  4. 10.]


In [ ]:
# id column
id = list(range(0,x_test.shape[0]))

In [ ]:
dict = {'id': id, 'label': y_pred}  
       
result = pd.DataFrame(dict) 
    
# saving the dataframe 
result.to_csv('file.csv', index=False)

In [ ]:
from google.colab import files
files.download("file.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>